# --- CORRELATING (FLUX OF) SPOTS (FAKE STARS FROM SPOT PROJECTOR) WITH FLAT FIELDS ---

The spot projector is used is 2 ways:
        taking many images at one location on the sensor
        taking many images each at a different position on the sensor
            this type of data we have for one ITL and one E2V sensor 

Looking at the relative throughput of each pixel in the sensor, checking if it is the same when tested with the spot projector vs. flat fielding. Answers/confirms the preceding question of: does flat fielding correctly describe how QE varies per pixel per wavelength?? 

If there are variations in relative throughput between spots (for one sensor), this can tell us how much of the variation is due to the flux of the star/light source vs. due to the response of the CCD OR position of sensor/amplifier/pixel in the FOCAL PLANE! (QE can vary spatially depending on region of focal plane/sensor/amp)

Calibrate average flux value for spots and for CCOB flat. Divide each "star" and each CCOB flat pixel by the respective average flux value to get the CALIBRATED value of each "star"/flat pixel.

I will access Adam's code and pull flux data (with mean of flux of each star, one mean for each dither) and x,y position data to tell me where the star is on the grid. This mean flux is our "calibration" value, I can take the flux of each star and divide by the calibration value to get relative throughput for each "star" (RESIDUAL) and see how this varies with focal plane location, etc., then I can compare this calibrated value in a "star" region against the same positional region in the flat. It will be important to use code that conducts mapping between amp/ccd/focal plane coordinates.

(https://github.com/snyder18/mixcoatl)

#### "STARS" SHOULD IDEALLY SHOW THE SAME VARIATIONS THAT THE FLAT FIELDS DO!!

#### Yousuke's idea:

- Yes. the one Oscar is working but complimentary. I am not super clear yet but let me try to explain. As we don't know how the QE variation affects in science product, investigating the correlation between QE variation -- flat with spot is interesting. Thinking about the spot, you have measurements of flux (zero-th), positions (1st), and shape (2nd moments). So you can correlate those measurements with QE variations. Suppose we are going to compare flux f a function of positions against QE variation Q (as a function of positions), we can calculate correlation coefficient r =<df dQ>/(<df^2><dQ^2>) using deviations df, dQ from mean numbers <f>, <Q> . 
- If there is the correlation, r will be a non-zero value and vice versa. You could repeat this analysis with positional differences from fiducial as a function of positions, shapes as a function of positions.
But what I am not clear is how we say whether if it is significant or not. We should evaluate noise level in r but I haven't had a clear view how we evaluate noise level, but eventually I'll come up with some idea hopefully? Probably we could use typical methods -- bootstrap or jackknife but I am not super clear. 

In [1]:
# all 1400? position data filed located in spotdata directory
# referencing Adam's code on git to utilize data, changing it to measure flux though (not centroid)

# NaN : not a number, a value in an array such as 0/0 is NaN
# masked array : missing data values

In [2]:
# sourceGridTask.py imports functions in sourcegrid.py

"""
import os
import numpy as np
from os.path import join, splitext
import scipy
from scipy.spatial import distance
from astropy.io import fits
import lsst.pex.config as pexConfig
import lsst.pipe.base as pipeBase
from lsst.obs.lsst import LsstCamMapper as camMapper
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
"""

from sourcegrid import BaseGrid, DistortedGrid, grid_fit, coordinate_distances
from sourceGridTask import SourceGridConfig, SourceGridTask

## I'll want to build a "calibration" file, holding the mean flux for each artificial star.  Adam has something similar now for the centroid - thats the gX and gY in the code I think. You’ll need to make mean flux values and subtract those off when calculating dflux_array.


From Adam's notebook:

In [3]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import glob

In [4]:
## Example processed grid, all of these files are located in my directory: spotdata
## 3rd file down, testing
infile = '/home/lpayne22/spotdata/MC_C_20191030_000363_R22_S11_15.0s_0.1x_-3.5y_processed_grid.fits'
#infile = '/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/BOT/6864D_dither/results/MC_C_20191030_000363_R22_S11_15.0s_0.1x_-3.5y_processed_grid.fits'

In [5]:
## Open as a FITs table with all of spot data using Astropy
hdul = fits.open(infile)

## Print Main Header
print('Main Header\n')
print(repr(hdul[0].header))


## Print Table Header
print('\nTable Header\n')
print(repr(hdul[1].header))

Main Header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
X0      =    2385.220108337066                                                  
Y0      =    1996.675045679444                                                  
XSTEP   =    65.34408630180397                                                  
YSTEP   =    65.35191691653706                                                  
THETA   = -0.05426371615038783                                                  
NCOLS   =                   49                                                  
NROWS   =                   49                                                  

Table Header

XTENSION= 'BINTABLE'           / binary table extension                         


In [6]:
## Get Data table
data = hdul[1].data

## Get fake stars x/y positions and XX moment
x = data['X']
y = data['Y']
xx = data['XX']
yy = data['YY']

width = np.sqrt(xx**2.+yy**2.)

## For each star plot its mean 'width' as defined above (calculated from many measurements of each star)
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
im = ax.scatter(x, y, c=width)

fig.colorbar(im)
ax.set_title('Fake Star Widths', fontsize=20)

Text(0.5, 1.0, 'Fake Star Widths')

In [7]:
## Get Data table
data = hdul[1].data

## Get fake stars x/y positions and XX moment
x = data['X']
y = data['Y']

flux = data['FLUX']

## For each star its flux is plotted
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
im = ax.scatter(x, y, c=flux)

fig.colorbar(im)
ax.set_title('Fake Star Fluxes', fontsize=20)

Text(0.5, 1.0, 'Fake Star Fluxes')